In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import itertools
import collections
warnings.filterwarnings("ignore")

plt.style.use("seaborn")
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Работа с данными и визуализация

In [2]:
df = pd.read_csv('Алкопарсингновый.csv')

# заранее сделал копию изначального DataFrame, чтобы сохранились данные без 95% квантиля
df_without_quantile = df

## Описание данных:

`name` - полное название вина

`w_id` - ID конретного вина на сайте SimpleWine

`href` - ссылка на вино на сайте

`base_price` - начальная цена бутылки вина (руб)

`discount_size` - предоставляемая скидка (%)

`discount_price` - скидочная цена бутылки вина

`simple_collection` - состоит ли бутылка вина в коллекции магазина SimpleWine

`country` - страна производства

`color` - цвет вина

`sweetness` - вкус вина

`volume` - объём бутылки

`SW_rating`, `VIVINO_rating` - клиентский рейтинг вин

`WS_rating`, `RVF_rating`, `JS_rating`, `RP_rating`, `AM_rating`, `GR_rating`, `AIS_rating`, `PENIN_rating`, `ST_rating`, `JR_rating` - рейтинги различных критиков

`grape` - сорт винограда

`manufacturerz` - производитель вина

`features` - дополнительные характеристиики вина

`region` - региона производства вина

`year` - год производства

`strength` - крепкость вина (содержание алкоголя)

`storage_potential` - срок годности

`appelltion` - гос стандарт для выращивания вина

`category` - категория вина

`decantation` - рекомендуется ли декантация вина

`aging_in_container` - где вино настаивалось

`taste` - вкус вина

`dishes` - к каким блюдам подается вино

`sugar` - содержание сахара (по 5-бальной шкале)

`acidity` - кислотность (по 5-бальной шкале)

`aromaticity` - ароматические свойства вина (по 5-бальной шкале)

`tannins` - таннины (по 5-бальной шкале)

`body` - тело вина (по 5-бальной шкале)

`description` - описание вина на сайте

# Часть 1

## Предобработка полученного DataFrame

In [3]:
# Спарсили по 45 признаков на 5512 бутылок вина
print(df.shape)

(5512, 45)


In [4]:
# заметим, что длина уникальных значений по колонкам 'w_id' и 'href' совпадают и равняются 4436, 
# что говорит нам о том, что в первоначальном DataFrame были несколько дубликатов
len(df['w_id'].unique()), len(df['href'].unique())

(4436, 4436)

In [5]:
# посчитаем количество повторяющихся строк
print('Количество дубликатов:', df.shape[0] - len(df['w_id'].unique()))

# удалим все повторяющиеся строки (дубликаты)
df.drop_duplicates(subset='w_id', keep='first', inplace=True)

Количество дубликатов: 1076


In [6]:
# заметим, что в столбце 'gift_wrap' только одно значение - False
df['gift_wrap'].value_counts()

# столбец 'gift_wrap'не несёт какой-либо смысловой нагрузки => удаляем
df.drop('gift_wrap', axis=1, inplace=True)

# столбец 'Unnamed: 0' не несёт никакой информации => удаляем
df.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
# в нашем DataFrame есть наборы вин (по 2-6 штук в каждом) => надо от них избавиться
nabor_and_set = df.loc[(df['name'].str.split().str[:2].str.join(' ') == 'Вино Набор') 
   | (df['name'].str.split().str[:2].str.join(' ') == 'Вино Set')].index

# удаляем все наборы из нашего DataFrame 
df.drop(index=nabor_and_set, inplace=True)

# удалим ещё парочку наборов, которые имеют нестандратное название 
dop_nabor_and_set = ['/catalog/product/brunello_di_montalcino_castelgiocondo__075_gift_124834/', 
                     '/catalog/product/nab_petrikor_aligote_petrikor_krasnoe_v_p_u_075_gift_140054/']
# (мы их нашли вручную и будем удалять из DataFrame по 'href')

Dop_nabor_and_set = df.loc[df['href'].isin(dop_nabor_and_set)].index
# удаляем наборы с нестандартными названиями
df.drop(index=Dop_nabor_and_set, inplace=True)

In [8]:
# в нашем ДатаФрейме есть не только вина
df['name'].str.split().str[0].value_counts()

Вино        4391
Херес         17
Портвейн      17
Name: name, dtype: int64

In [9]:
# создадим для удобства обработки данных новый столбец с типом алкогольных напитков
df['type'] = df['name'].str.split().str[0]

# удаляем Портвейны и Хересы
port_and_her = df.loc[(df['type'] == 'Портвейн') | (df['type'] == 'Херес')].index
df.drop(index=port_and_her, inplace=True)

In [10]:
# проверяем столбец 'type' на информативность
df['type'].value_counts()

# удаляем столбец 'type', поскольку уже не нужен
df.drop('type', axis=1, inplace=True)

In [11]:
# чистый однобутылочный винный DataFrame
print(df.shape)

(4391, 43)


# Часть 2

## Продолжаем обрабатывать данные и разбираемся с пропусками

In [12]:
# удаляем все признаки, где количество пропусков не превышает 50, поскольку сильно они ничего не поменяют
pystushki = ['color', 'sweetness', 'grape', 'manufacturer', 'region', 'strength', 'storage_potential', 'aging_in_container', 'taste']
for i in range(len(pystushki)):
    df.drop(index=df[df[pystushki[i]].isna()].index, inplace=True)
    
# удалим и оставшиеся 35 строк с пустыми значениями в столбце 'year'
df.drop(index=df[df['year'].isna()].index, inplace=True)

# заметим, что столбец 'no_alco' больше не имеет никакой смысловой нагрузаи => удаляем
df.drop('no_alco', axis=1, inplace=True)

print(df.shape)

(4267, 42)


In [13]:
# избавимся от слишком дорогих вин в нашем DataFrame (выбросы нам не нужны)
threshold = df['base_price'].quantile(0.95)
df = df[df['base_price'] <= threshold]

In [14]:
chto_to = ['decantation', 'sugar', 'acidity', 'aromaticity', 'tannins', 'body']
for i in range(len(chto_to)):
    print('Уникальные значения признака:', chto_to[i])
    if chto_to[i] != 'decantation':
        print(np.sort(df[chto_to[i]].unique())[::1])
    else:
        print(df[chto_to[i]].unique())
    print()

Уникальные значения признака: decantation
['не рекомендуется' 'декантация' 'аэрация' nan]

Уникальные значения признака: sugar
[ 1.  2.  3.  4.  5. nan]

Уникальные значения признака: acidity
[ 0.  1.  2.  3.  4.  5. nan]

Уникальные значения признака: aromaticity
[ 2.  3.  4.  5. nan]

Уникальные значения признака: tannins
[ 2.  3.  4.  5. nan]

Уникальные значения признака: body
[ 0.  1.  2.  3.  4.  5. nan]



In [15]:
# подумаем, что делать с рейтингами, их у нас 12 различных штук
ratings = ['SW_rating', 'VIVINO_rating', 'WS_rating', 'RVF_rating', 'JS_rating', 
           'RP_rating', 'AM_rating', 'GR_rating', 'AIS_rating', 'PENIN_rating', 
           'ST_rating', 'JR_rating']
rate = df[ratings]

# посчитаем какое количество оценок у вин
for i in range(1, len(ratings)):
    if i == 1:
        print('Количество бутылок с', i, 'рейтингом:', (rate.count(axis=1) == i).sum())
    else:
        if (rate.count(axis=1) == i).sum() != 0:
            print('Количество бутылок с', i, 'рейтингами:', len(rate[rate.isna().sum(axis=1) == (12 - i)]))
# это нам говорит о том, что максимум у вина может быть 6 различных оценок

# посмотрим сколько всего вин вообще никак не оценены (нет ни одного рейтинга)
print()
print('У', rate[rate.isna().all(axis=1)].shape[0], 'вин нет ни одного рейтинга')

Количество бутылок с 1 рейтингом: 1420
Количество бутылок с 2 рейтингами: 1509
Количество бутылок с 3 рейтингами: 435
Количество бутылок с 4 рейтингами: 196
Количество бутылок с 5 рейтингами: 54
Количество бутылок с 6 рейтингами: 1

У 439 вин нет ни одного рейтинга


In [16]:
for i in range(len(ratings)):
    print('Название рейтинга:', ratings[i])
    print(np.sort(rate[ratings[i]].unique())[::1])
    print('Количество уникальных значений -', len(rate[ratings[i]].unique()))
    print('Всего пропусков этого рейтинга -', rate[ratings[i]].isnull().sum())
    print()

Название рейтинга: SW_rating
[2.3 2.8 2.9 3.  3.2 3.3 3.4 3.5 3.7 3.8 3.9 4.  4.1 4.2 4.3 4.4 4.5 4.6
 4.7 4.8 4.9 5.  nan]
Количество уникальных значений - 23
Всего пропусков этого рейтинга - 1157

Название рейтинга: VIVINO_rating
[3.7 3.8 3.9 4.  4.1 4.2 4.3 4.4 4.5 4.6 4.7 nan]
Количество уникальных значений - 12
Всего пропусков этого рейтинга - 1816

Название рейтинга: WS_rating
[ 86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.  98.  99.
 100.  nan]
Количество уникальных значений - 16
Всего пропусков этого рейтинга - 3779

Название рейтинга: RVF_rating
[19. nan]
Количество уникальных значений - 2
Всего пропусков этого рейтинга - 4052

Название рейтинга: JS_rating
[ 89.  90.  91.  92.  93.  94.  95.  96.  97.  98.  99. 100.  nan]
Количество уникальных значений - 13
Всего пропусков этого рейтинга - 3408

Название рейтинга: RP_rating
[ 87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.  98.  99. 100.
  nan]
Количество уникальных значений - 15
Всего пропусков этого рейтин

**В нашем DataFrame всего 12 различных оценочных рейтингов. Мы их разделим на 2 группы:**
1. Рейнтиги критиков: WS_rating, RVF_rating, JS_rating, RP_rating, AM_rating, GR_rating, AIS_rating, PENIN_rating, ST_rating, JR_rating.
2. Рейтинги клиентов: SW_rating, VIVINO_rating

Возьмём средние оценки среди рейнтигов внутри каждой группы и создадим два новых столбца с полученными значениями

Чтобы мы могли посчитать корректное среднее значение по рейнтигам, то в первую очередь должны всё привести к единой системе измерения. Поскольку в нашем DataFrame есть 5-бальные, 20-бальные и 100-бальные рейтинги

In [17]:
# для рейнтига критиков - переводим в 100-бальную систему
df['RVF_rating'] *= 5
df['GR_rating'] /= 3
df['GR_rating'] *= 100
df['AIS_rating'] *= 20
df['JR_rating'] *= 5

# для клиентского рейнтига - переводим в 100-бальную систему
df['SW_rating'] *= 20
df['VIVINO_rating'] *= 20

# создадим два новых столбца с клиентким рейтингом и рейнтигом критиков
df['client_rating'] = df[['SW_rating', 'VIVINO_rating']].mean(axis=1)
df['critic_rating'] = df[['WS_rating', 'RVF_rating', 'JS_rating', 
           'RP_rating', 'AM_rating', 'GR_rating', 'AIS_rating', 'PENIN_rating', 
           'ST_rating', 'JR_rating']].mean(axis=1)

In [18]:
# создадим новый столбик с значением модуля рахности клиентского рейтинга и рейнтига критиков
df['otklonenue'] = abs(df['client_rating'] - df['critic_rating'])

df[['client_rating', 'critic_rating', 'otklonenue']].agg(['mean', 'median', 'min', 'max'])

,client_rating,critic_rating,otklonenue
mean,88.012069,92.579751,6.343279
median,88.000000,92.333333,6.000000
min,46.000000,60.000000,0.000000
max,100.000000,100.000000,27.000000


In [19]:
# некоторые признаки, которые должны быть списками, превратились в строки, исправим
for c in ['grape', 'features', 'taste', 'dishes', 'description']:
    df[c][df[c].isnull() == False] = df[c][df[c].isnull() == False].apply(eval)

In [20]:
# разделим английские слова, которые не разделились при парсинге
for i in range(len(df['features'])):
    if df['features'].iloc[i] == ['GreenselectionSustainable']:
        df['features'].iloc[i] = re.findall('[A-Z][^A-Z]*', df['features'].iloc[i][0])

In [21]:
# уберем процентное содержание винограда и превратим в список
for i in range(len(df['grape'])):
    pattern = r'[0-9]'
    df['grape'].iloc[i] = [re.sub(pattern, '', s).replace('%','').strip() for s in df['grape'].iloc[i]]

In [22]:
#найдем самый популярный виноград для каждой страны
most_popular_grape = {}
for c in df['country'].values:
    listik = list(df['grape'][df['country']==c])
    flat_listik = list(itertools.chain(*listik))
    most_popular_grape[c] = max(collections.Counter(flat_listik), key=collections.Counter(flat_listik).get)

# Часть 3

## Виузализация DataFrame

Мы нашли интересную визуализацию данных в виде карты мира, которая сразу предоставит читателю полную картину по данным, которые мы спарсили. Мы постарались вынести на карту самые информативыне показатели нашего DataFrame.

Но перед тем, как его реализовать необходимо подготовить все данные

In [23]:
# сгруппировали данные по странам и вывели некоторые статистические наблюдения
metrics_by_country = df.groupby(['country'])['base_price'].agg(['count', 'mean', 'median', 'min', 'max']
                                         ).round().reset_index().sort_values('country')
# мы сразу же отсортировали этот DataFrame по странам в алфавитном порядке, 
# чтобы в дальнейшем не было путаниц c очередностью стран (а так они стоят в алфавитном порядке)
# раположили в алфавитном порядке и сделали список из всех представленных стран

In [24]:
# cоздадим словарь с кодами стран, состоящие из 3ёх заглавных букв (выставленные в алфавитном порядке на рус языке)
сountry_code = ['AUS', 'AUT', 'ARG', 'ARM', 'HUN', 'DEU', 'GRC', 'GEO', 
                'ISR', 'ESP', 'ITA', 'NZL', 'PRT', 'MDA', 'RUS', 'SRB', 
                'SVN', 'USA', 'FRA', 'CHL', 'ZAF']

# сразу же добавим ко всем показателям
metrics_by_country.insert(0, 'CODE', сountry_code)

# мы перевели страны в коды, чтобы найденная нами визуализация заработала, 
# поскольку она работает именно на этих кодах

In [25]:
# добавим значения самых дорогих вин из DataFrame без квантиля
expensive_wines = df_without_quantile.groupby(['country'])['base_price'].agg(['max']).reset_index().sort_values('country')
# обязательно отсортировали по алфавиту, чтобы в итоговой таблице все значения находились на своих местах

# добавляем ко всем показателям
metrics_by_country['MAX'] = expensive_wines['max']
# и удаляем максимальыне значения при 95% квантиле
metrics_by_country.drop('max', axis=1, inplace=True)

In [26]:
# чуть позже мы получили словарь с самыми популярными видами винограда в каждой стране (most_popular_grape)
# чуть поменяем его вид, чтобы было удобнее работать
popular_grape = {'country': ['Франция', 'Португалия', 'Россия', 'Испания', 'Италия', 'Германия', 'Аргентина', 'Южная Африка', 'Австрия', 'Грузия', 'Чили', 'Австралия', 'Израиль', 'Соединенные Штаты Америки', 'Новая Зеландия', 'Сербия', 'Греция', 'Венгрия', 'Республика Молдова', 'Армения', 'Словения'],
        'grape': ['мерло', 'турига насьональ', 'каберне совиньон', 'темпранильо', 'санджовезе', 'рислинг', 'шардоне', 'шенен блан', 'грюнер вельтлинер', 'саперави', 'каберне совиньон', 'шираз', 'каберне совиньон', 'каберне совиньон', 'пино нуар', 'шардоне', 'мерло', 'фурминт', 'каберне совиньон', 'арени', 'фурминт']}

# создаем ещё один дата фрейм и обязательно сортируем страны по алфавиту
grape = pd.DataFrame(popular_grape).sort_values('country')

# добавляем ко всем показателям
metrics_by_country['grape'] = grape['grape']

In [27]:
# найдем самый распространненый цвет вина по странам
spisok_favorite_color = []
for i in range(len(metrics_by_country['country'])):
    favorite = df[df['country'] == metrics_by_country['country'][i]
                  ]['color'].value_counts().to_frame().reset_index()['index'][0]
    spisok_favorite_color.append(favorite)

# добавляем ко всем показателям
metrics_by_country['fav_color'] = spisok_favorite_color

In [28]:
# найдем самых популярных производителей вина в каждой стране
spisok_popular_manufacturer = []
for i in range(len(metrics_by_country['country'])):
    popular = df[df['country'] == metrics_by_country['country'][i]
                  ]['manufacturer'].value_counts().to_frame().reset_index()['index'][0]
    spisok_popular_manufacturer.append(popular)

# добавляем ко всем показателям
metrics_by_country['popular_manufacturer'] = spisok_popular_manufacturer

# избавимся от float
metrics_by_country['mean'] = metrics_by_country['mean'].astype(int)
# Наш DataFrame со статистическими показателями по всем странам готов

In [29]:
# подготовим данные, которые хотим показать при визуализации
metrics_by_country['info'] = (
    metrics_by_country['country'] + '<br>' +
    'Кол-во вин: ' + metrics_by_country['count'].astype(str) + '<br>' +
    'Самое дешёвое вино: ' + metrics_by_country['min'].astype(int).astype(str) + '<br>' +
    'Самое дорогое вино: ' + metrics_by_country['MAX'].astype(str) + '<br>' +
    'Самый популярный цвет вина: ' + metrics_by_country['fav_color'].astype(str) + '<br>' +
    'Самый популярный сорт винограда: ' + metrics_by_country['grape'].astype(str) + '<br>' +
    'Самый популярный производитель: ' + metrics_by_country['popular_manufacturer'].astype(str)
)

In [35]:
import plotly.graph_objects as go
import plotly.io as pio

fig = go.Figure(data=go.Choropleth(
    locations = metrics_by_country['CODE'],
    z = metrics_by_country['mean'],
    text = metrics_by_country['info'],
    colorscale = 'Reds',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='black',
    marker_line_width=0.7,

))

fig.update_layout(
    title_text='Средняя стоимость бутылки вина',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Data Тоскана',
        showarrow = False
    )]
)

fig.show()

In [39]:
#экспортируем карту
pio.write_html(fig, 'map.html')

Данная визуализация данных представляет собой хороплет-карту, которая отображает среднюю стоимость бутылки вина в различных странах. Она полезна и интересна по нескольким причинам:

**Географическое распределение:** Карта позволяет легко увидеть, как стоимость вина различается в разных странах. Это может быть полезно для анализа рынков и понимания, где вина обычно дороже или дешевле.

**Цветовая шкала:** Цветовая шкала используется для отображения разных уровней стоимости вина. Более темные оттенки цвета указывают на более высокую стоимость, а более светлые оттенки - на более низкую стоимость. Это позволяет быстро сравнить и оценить относительную стоимость в разных странах.

**Интерактивность:** Поскольку график создается с использованием библиотеки Plotly, он обладает интерактивными возможностями. Пользователь может навести курсор на конкретную страну, чтобы узнать ее среднюю стоимость вина и другую информацию, такую как минимальную и максимальную стоимость бутылки в стране, самый популярный сорт винограда, самый популярный произвдитель и так далее. Это добавляет удобство и взаимодействие с данными.

**Визуальный эффект:** Хороплет-карты обычно очень привлекательны визуально, поскольку они позволяют наглядно представить данные на географической карте. Это делает визуализацию более привлекательной и интересной для зрителя.

P.s. Остальная визуализация содержится в тетрадках с гипотезами

P.p.s. Если у вас не прогрузится хороплет-карта, вы сможете найти её версию html в нашем репозитории

In [40]:
#выгрузим обработанные данные
df.to_csv('обработанные данные.csv')